In [11]:
import pandas as pd

In [12]:
raw_19_df = pd.read_excel(
    "./../raw_data/제_19_대통령선거_개표.xlsx",
    header=[0, 1],
)

In [13]:
df = raw_19_df.copy()

In [14]:
df.head(3)

시도명               구시군명               읍면동명               투표구명  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
0                 전국                NaN                NaN                NaN   
1              서울특별시                 합계                NaN                NaN   
2              서울특별시                종로구                 합계                NaN   

                선거인수                투표수    후보자별 득표수                       \
  Unnamed: 4_level_1 Unnamed: 5_level_1 더불어민주당\n문재인 자유한국당\n홍준표 국민의당\n안철수   
0           42479710           32807908    13423800    7852849   6998342   
1            8382999            6590646     2781345    1365285   1492767   
2             133769             102566       42512      22325     22313   

             ...                                                           \
  바른정당\n유승민  ... 경제애국당\n오영국 국민대통합당\n장성민 늘푸른한국당\n이재오 민중연합당\n김선동 한국국민당\n이경희   
0   2208771  ...       6040       21709        9140      27229      11355   
1    476973  ...        789        3554        1938       3416       1277   
2      7412  ...          5          78          31         63         26   

                                            무효투표수                 기권수  
  홍익당\n윤홍식 무소속\n김민찬         계 Unnamed: 20_level_1 Unnamed: 21_level_1  
0    18543    33990  32672175              135733             9671802  
1     2177     3950   6568917               21729             1792353  
2       47       49    102202                 364               31203  

[3 rows x 22 columns]

In [15]:
# preproc columns
_replace_unnamed = lambda x: "" if x.startswith("Unnamed") else x
_replace_escape = lambda x: x.replace("\n", "_")
_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace_escape, map(_replace_unnamed, x)))

df.columns = list(map(func, df.columns))
df.head(3)

,시도명,구시군명,읍면동명,투표구명,선거인수,투표수,후보자별 득표수_더불어민주당_문재인,후보자별 득표수_자유한국당_홍준표,후보자별 득표수_국민의당_안철수,후보자별 득표수_바른정당_유승민,...,후보자별 득표수_경제애국당_오영국,후보자별 득표수_국민대통합당_장성민,후보자별 득표수_늘푸른한국당_이재오,후보자별 득표수_민중연합당_김선동,후보자별 득표수_한국국민당_이경희,후보자별 득표수_홍익당_윤홍식,후보자별 득표수_무소속_김민찬,후보자별 득표수_계,무효투표수,기권수
0,전국,NaN,NaN,NaN,42479710,32807908,13423800,7852849,6998342,2208771,...,6040,21709,9140,27229,11355,18543,33990,32672175,135733,9671802
1,서울특별시,합계,NaN,NaN,8382999,6590646,2781345,1365285,1492767,476973,...,789,3554,1938,3416,1277,2177,3950,6568917,21729,1792353
2,서울특별시,종로구,합계,NaN,133769,102566,42512,22325,22313,7412,...,5,78,31,63,26,47,49,102202,364,31203


In [16]:
df = df[
    (df["시도명"] != "전국")
    & (df["구시군명"] != "합계")
    & ~(
        df["읍면동명"].isin(
            ["합계", "거소·선상투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
        )
    )
    & (df["투표구명"] == "합계")
].copy()
df.head(3)

,시도명,구시군명,읍면동명,투표구명,선거인수,투표수,후보자별 득표수_더불어민주당_문재인,후보자별 득표수_자유한국당_홍준표,후보자별 득표수_국민의당_안철수,후보자별 득표수_바른정당_유승민,...,후보자별 득표수_경제애국당_오영국,후보자별 득표수_국민대통합당_장성민,후보자별 득표수_늘푸른한국당_이재오,후보자별 득표수_민중연합당_김선동,후보자별 득표수_한국국민당_이경희,후보자별 득표수_홍익당_윤홍식,후보자별 득표수_무소속_김민찬,후보자별 득표수_계,무효투표수,기권수
6,서울특별시,종로구,청운효자동,합계,9378,7295,3109,1677,1438,475,...,0,11,1,6,1,5,6,7273,22,2083
12,서울특별시,종로구,사직동,합계,7613,5788,2146,1617,1161,455,...,0,11,0,1,0,2,3,5770,18,1825
15,서울특별시,종로구,삼청동,합계,2271,1654,617,456,316,113,...,0,1,2,1,2,0,0,1650,4,617


In [17]:
# renaming

rename_dict = {
    "시도명": "loc_1",
    "구시군명": "loc_2",
    "읍면동명": "loc_3",
}

df.rename(columns=rename_dict, inplace=True)

_replace_default = lambda x: x.replace("후보자별 득표수_", "")
df.columns = list(map(_replace_default, df.columns))
df.head(3)

,loc_1,loc_2,loc_3,투표구명,선거인수,투표수,더불어민주당_문재인,자유한국당_홍준표,국민의당_안철수,바른정당_유승민,...,경제애국당_오영국,국민대통합당_장성민,늘푸른한국당_이재오,민중연합당_김선동,한국국민당_이경희,홍익당_윤홍식,무소속_김민찬,계,무효투표수,기권수
6,서울특별시,종로구,청운효자동,합계,9378,7295,3109,1677,1438,475,...,0,11,1,6,1,5,6,7273,22,2083
12,서울특별시,종로구,사직동,합계,7613,5788,2146,1617,1161,455,...,0,11,0,1,0,2,3,5770,18,1825
15,서울특별시,종로구,삼청동,합계,2271,1654,617,456,316,113,...,0,1,2,1,2,0,0,1650,4,617


In [18]:
df.columns

Index(['loc_1', 'loc_2', 'loc_3', '투표구명', '선거인수', '투표수', '더불어민주당_문재인',
       '자유한국당_홍준표', '국민의당_안철수', '바른정당_유승민', '정의당_심상정', '새누리당_조원진', '경제애국당_오영국',
       '국민대통합당_장성민', '늘푸른한국당_이재오', '민중연합당_김선동', '한국국민당_이경희', '홍익당_윤홍식',
       '무소속_김민찬', '계', '무효투표수', '기권수'],
      dtype='object')

In [19]:
def replace_position(df, lib, con, etc):
    """
    lib : 진보당
    con : 보수당
    etc : 나머지당
    """
    df["lib"] = df.loc[:, lib].sum(axis=1)
    df["con"] = df.loc[:, con].sum(axis=1)
    df["etc"] = df.loc[:, etc].sum(axis=1)
    df.drop(columns=(lib + con + etc), inplace=True)
    return df


lib = ["더불어민주당_문재인"]
con = ["자유한국당_홍준표"]
etc = [
    "국민의당_안철수",
    "바른정당_유승민",
    "정의당_심상정",
    "새누리당_조원진",
    "경제애국당_오영국",
    "국민대통합당_장성민",
    "늘푸른한국당_이재오",
    "민중연합당_김선동",
    "한국국민당_이경희",
    "홍익당_윤홍식",
    "무소속_김민찬",
]
df = replace_position(df, lib, con, etc)
df.head(3)

,loc_1,loc_2,loc_3,투표구명,선거인수,투표수,계,무효투표수,기권수,lib,con,etc
6,서울특별시,종로구,청운효자동,합계,9378,7295,7273,22,2083,3109,1677,2487
12,서울특별시,종로구,사직동,합계,7613,5788,5770,18,1825,2146,1617,2007
15,서울특별시,종로구,삼청동,합계,2271,1654,1650,4,617,617,456,577


In [20]:
df.drop(columns=["투표구명"], inplace=True)

In [21]:
대통령_19 = df.copy()
대통령_19.head(5)

,loc_1,loc_2,loc_3,선거인수,투표수,계,무효투표수,기권수,lib,con,etc
6,서울특별시,종로구,청운효자동,9378,7295,7273,22,2083,3109,1677,2487
12,서울특별시,종로구,사직동,7613,5788,5770,18,1825,2146,1617,2007
15,서울특별시,종로구,삼청동,2271,1654,1650,4,617,617,456,577
19,서울특별시,종로구,부암동,7702,5916,5904,12,1786,2354,1502,2048
23,서울특별시,종로구,평창동,14118,10881,10844,37,3237,3762,3362,3720


In [22]:
대통령_19.to_csv("./1st_preproc_data/대통령_19.csv")

In [23]:
raw_20_df = pd.read_excel("./../raw_data/제_20_대통령선거_개표.xlsx", header=[0], thousands=",")

In [24]:
df = raw_20_df.copy()

In [25]:
df.head(3)

,시도,구시군,읍면동명,투표구명,선거인수,투표수,더불어민주당\n이재명,국민의힘\n윤석열,정의당\n심상정,기본소득당\n오준호,...,노동당\n이백윤,새누리당\n옥은호,신자유민주연합\n김경재,우리공화당\n조원진,진보당\n김재연,통일한국당\n이경희,한류연합당\n김민찬,계,무효투표수,기권수
0,전국,합계,NaN,NaN,44197692,34067853,16147738,16394815,803358,18105,...,9176,4970,8317,25972,37366,11708,17305,33760311,307542,10129839
1,서울특별시,합계(특별시),NaN,NaN,8346647,6501831,2944981,3255747,180324,3829,...,1571,844,1791,4657,5615,1333,1907,6439139,62692,1844816
2,서울특별시,종로구,합계,NaN,129968,100629,46130,49172,3115,53,...,42,9,50,82,105,20,30,99366,1263,29339


In [26]:
# preproc columns

_replace_escape = lambda x: x.replace("\n", "_")
df.columns = list(map(_replace_escape, df.columns))
df.head(3)

,시도,구시군,읍면동명,투표구명,선거인수,투표수,더불어민주당_이재명,국민의힘_윤석열,정의당_심상정,기본소득당_오준호,...,노동당_이백윤,새누리당_옥은호,신자유민주연합_김경재,우리공화당_조원진,진보당_김재연,통일한국당_이경희,한류연합당_김민찬,계,무효투표수,기권수
0,전국,합계,NaN,NaN,44197692,34067853,16147738,16394815,803358,18105,...,9176,4970,8317,25972,37366,11708,17305,33760311,307542,10129839
1,서울특별시,합계(특별시),NaN,NaN,8346647,6501831,2944981,3255747,180324,3829,...,1571,844,1791,4657,5615,1333,1907,6439139,62692,1844816
2,서울특별시,종로구,합계,NaN,129968,100629,46130,49172,3115,53,...,42,9,50,82,105,20,30,99366,1263,29339


In [27]:
# filter agg / etcs

df = df[
    (df["시도"] != "전국")
    & ~(df["구시군"].isin(["합계", "합계(광역시)", "합계(도)", "합계(특별시)"]))
    & ~(
        df["읍면동명"].isin(
            ["합계", "거소·선상투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
        )
    )
    & (df["투표구명"] == "소계")
].copy()
df.head(3)

,시도,구시군,읍면동명,투표구명,선거인수,투표수,더불어민주당_이재명,국민의힘_윤석열,정의당_심상정,기본소득당_오준호,...,노동당_이백윤,새누리당_옥은호,신자유민주연합_김경재,우리공화당_조원진,진보당_김재연,통일한국당_이경희,한류연합당_김민찬,계,무효투표수,기권수
7,서울특별시,종로구,청운효자동,소계,8706,6640,3150,3137,237,3,...,2,0,3,7,4,1,2,6572,68,2066
12,서울특별시,종로구,사직동,소계,7651,5850,2260,3282,212,3,...,0,1,2,4,4,0,0,5790,60,1801
16,서울특별시,종로구,삼청동,소계,1935,1443,591,767,51,0,...,0,0,3,3,0,0,0,1425,18,492


In [28]:
# renaming

rename_dict = {
    "시도": "loc_1",
    "구시군": "loc_2",
    "읍면동명": "loc_3",
}
df.rename(columns=rename_dict, inplace=True)
df.drop(columns=["계"], inplace=True)
df.head(3)

,loc_1,loc_2,loc_3,투표구명,선거인수,투표수,더불어민주당_이재명,국민의힘_윤석열,정의당_심상정,기본소득당_오준호,국가혁명당 _허경영,노동당_이백윤,새누리당_옥은호,신자유민주연합_김경재,우리공화당_조원진,진보당_김재연,통일한국당_이경희,한류연합당_김민찬,무효투표수,기권수
7,서울특별시,종로구,청운효자동,소계,8706,6640,3150,3137,237,3,26,2,0,3,7,4,1,2,68,2066
12,서울특별시,종로구,사직동,소계,7651,5850,2260,3282,212,3,22,0,1,2,4,4,0,0,60,1801
16,서울특별시,종로구,삼청동,소계,1935,1443,591,767,51,0,10,0,0,3,3,0,0,0,18,492


In [29]:
lib = ["더불어민주당_이재명"]
con = ["국민의힘_윤석열"]
etc = [
    "정의당_심상정",
    "기본소득당_오준호",
    "국가혁명당 _허경영",
    "노동당_이백윤",
    "새누리당_옥은호",
    "신자유민주연합_김경재",
    "우리공화당_조원진",
    "진보당_김재연",
    "통일한국당_이경희",
    "한류연합당_김민찬",
]
df = replace_position(df, lib, con, etc)
df.head(3)

,loc_1,loc_2,loc_3,투표구명,선거인수,투표수,무효투표수,기권수,lib,con,etc
7,서울특별시,종로구,청운효자동,소계,8706,6640,68,2066,3150,3137,285
12,서울특별시,종로구,사직동,소계,7651,5850,60,1801,2260,3282,248
16,서울특별시,종로구,삼청동,소계,1935,1443,18,492,591,767,67


In [30]:
df.drop(columns=["투표구명"], inplace=True)

In [31]:
대통령_20 = df.copy()
대통령_20.head(10)

,loc_1,loc_2,loc_3,선거인수,투표수,무효투표수,기권수,lib,con,etc
7,서울특별시,종로구,청운효자동,8706,6640,68,2066,3150,3137,285
12,서울특별시,종로구,사직동,7651,5850,60,1801,2260,3282,248
16,서울특별시,종로구,삼청동,1935,1443,18,492,591,767,67
19,서울특별시,종로구,부암동,7205,5485,63,1720,2496,2731,195
24,서울특별시,종로구,평창동,13796,10534,79,3262,3866,6248,341
31,서울특별시,종로구,무악동,6143,4978,45,1165,2162,2586,185
35,서울특별시,종로구,교남동,7795,6215,68,1580,2621,3276,250
40,서울특별시,종로구,가회동,3454,2650,27,804,1252,1243,128
43,서울특별시,종로구,종로1·2·3·4가동,5741,3527,37,2214,1468,1911,111
47,서울특별시,종로구,종로5·6가동,4130,2906,37,1224,1305,1460,104


In [32]:
대통령_20.to_csv("./1st_preproc_data/대통령_20.csv")